In [2]:
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt


In [3]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [4]:
cd drive/MyDrive/

/content/drive/MyDrive


In [35]:
# ls

In [5]:
from dlc_practical_prologue import *

#Download Data

In [6]:
train_x, train_y, test_x, test_y = load_data(cifar = False, one_hot_labels = True, normalize = True, flatten = True)


* Using MNIST


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [36]:
train_x.shape[1], train_y.shape[1]

(784, 10)

##1- Activation functions

## Sigmoid

In [9]:
def sigma(x):
  return 1 / (1 + torch.exp(-x))


def dsigma(x):
  """
  compute the derivative sigmoid
  """
  return sigma(x) * (1 - sigma(x))

## Scaled sigmoid

In [14]:
def sigmoid(x):
  return (4 / (1 + torch.exp(-x))) - 2


def dsigmoid(x):
  """
  compute the derivative sigmoid
  """
  return 4 * sigmoid(x) * (1 - sigmoid(x))

## Tanh

In [19]:
def tanh(x):

  return torch.tanh(x)


def dtanh(x):
  """
  compute the derivative of tanh function
  """
  return 1 - torch.square(torch.tanh(x))

## Penalized tanh

In [32]:
def tanh(x , a):
  return torch.where(x>0, torch.tanh(x), a*torch.tanh(x))

def dtanh(x, a):

  """
  compute the derivative penalized tanh function
  """
  t = torch.tanh(x)
  return torch.where(x>0, 1 - t**2, a*(1 - t**2))

## ReLU

In [21]:
def Relu(x):
  return x*(x > 0)

def dRelu(x):
  """
  compute the derivative of ReLU function
  """
  return 1*(x >0)

## Leaky ReLU

In [23]:
def leakyReLU(x, a):
  return torch.where(x>0, x, a * x)

def dleakyReLU(x, a):
  """
  compute the derivative of leaky ReLU function
  """
  return torch.where(x > 0, 1., a)

In [25]:
t1 = torch.tensor([1,2,-30000., -5, -1, 1])
dleakyReLU(t1, 0.25)

tensor([1.0000, 1.0000, 0.2500, 0.2500, 0.2500, 1.0000])

## Loss

In [59]:
def loss(t , v):
  """
  computes the loss 
  """
  return torch.sum((t - v)**2)

def dloss(t, v):
  """
  computes the derivative 
  of loss 
  """
  return 2 * (v - t)

In [50]:
def forward_pass(w1, b1, w2, b2, x):
  """
  implements the feedforward pass for two layers
  """

  z1 = torch.mm(w1, x) + b1
  a1 = tanh(z1, 0.25)
  z2 = torch.mm(w2,a1) + b2
  a2 = tanh(z2, 0.25)
  return z1,a1,z2,a2



def backward_passs(w1, b1, w2, b2,t, x, s1, x1, s2, x2, d1_dw1, d1_db1, d1_dw2, d1_db2):
  """
  implements the backforward pass for two layers
  """

  learning_rate = 0.1
  N = x.shape[0]
  b1 -= 1/N * learning_rate * d1_db1
  b2 -= 1/N * learning_rate * d1_db2

  w1 -= 1/N * learning_rate * d1_dw1
  w2 -= 1/N * learning_rate * d1_dw2

  return w1,b1,w2,b2


In [51]:
X = train_x.t()
y = train_y.t() * 0.9
tst_x = torch.tensor(test_x).t()
tst_y = torch.tensor(test_y).t() * 0.9

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [52]:
import progressbar 

In [58]:
hidden_layer = 50

In [55]:
def train_func():

  '''
    Initialize the parameters:
  '''

  w1 = torch.empty(hidden_layer, train_x.shape[1]).normal_(mean = 0, std=1e-6)
  b1 = torch.empty(hidden_layer).normal_(mean = 0, std=1e-6).reshape(-1,1)
  w2 = torch.empty(train_y.shape[1], hidden_layer,).normal_(mean = 0, std=1e-6)
  b2 = torch.empty(train_y.shape[1]).normal_(mean = 0, std=1e-6).reshape(-1,1)

  dw1 = torch.empty(hidden_layer, train_x.shape[1])
  db1 = torch.empty(hidden_layer).reshape(-1,1)
  dw1 = torch.empty(hidden_layer, train_x.shape[1])
  db2 = torch.empty(train_y.shape[1]).reshape(-1,1)


  max_iter = 800
  train_errors = []
  test_errors  = [] 
  for i in range(max_iter):
    z1, a1, z2, a2 = forward_pass(w1,b1,w2,b2,X)

    
    da2 = dloss(y, a2)
    dz2 = da2 * dtanh(z2, 0.25)

    da1 = w2.t().mm(dz2)
    dz1 = da1 * dtanh(z1, 0.25)

    dw2 = dz2.mm(a1.t())
    dw1 = dz1.mm(X.t())
    db2 = dz2.sum(dim = 1).reshape(-1,1)
    db1 = dz1.sum(dim = 1).reshape(-1,1)
    
    w1,b1,w2,b2 = backward_passs(w1, b1, w2, b2,y, X, z1, a1, z2, a2, dw1, db1, dw2, db2)

    _,_,_,train_predictions = forward_pass(w1,b1,w2,b2,X)
    _,_,_,test_predictions  = forward_pass(w1,b1,w2,b2,tst_x)

    Train_Accu = torch.sum(train_predictions.argmax(dim=0) == y.argmax(dim=0)) / y.shape[1] * 100
    Test_Accu = torch.sum(test_predictions.argmax(dim=0) == t_y.argmax(dim=0)) / t_y.shape[1]  * 100  
    
    print("############## STOP #############")
    print("Train_Accuracy at step {} : {}".format(i, Train_Accu))
    print("Test_Accuracy error at step {} : {}".format(i, Test_Accu))
    





In [57]:
train_func()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,10), dpi = 40)
plt.plot(train_losses)
plt.plot(test_losses)
plt.legend(["Train Loss","Test Loss"])
plt.xlabel("Number of Iteration")
plt.ylabel("Loss")
plt.title("Train Loss vs Test Loss")
plt.show()